# module

In [ ]:
from joblib import Parallel, delayed
import os
os.environ['R_HOME'] = '/public/home/liuzj/softwares/anaconda3/envs/circadian/lib/R'
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from jpy_tools.rTools import py2r, r2py, r_inline_plot, rHelp, trl, rSet, rGet, ad2so, so2ad, so2md, rcontext, Trl, r2py_re, py2r_re
from jpy_tools import loadPkl, toPkl
rBase = importr('base')
rUtils = importr('utils')
dplyr = importr('dplyr')
reticulate = importr('reticulate')
R = ro.r
T = Trl()
R("options(browser='firefox', shiny.port=6533, max.print=200)")
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
import os

In [ ]:
os.listdir()

['01_circadian_clustering.ipynb',
 '01.fine_cluster_circadian_clustering.ipynb',
 '02_call_circadian.ipynb',
 '03.scenic.ipynb',
 '01.1.subclustering_c1.ipynb',
 'utils.py',
 '__pycache__',
 '04.celloracle.ipynb',
 'links.celloracle.links',
 'links.subsample.40000.celloracle.links',
 '01.0.benchmark.ipynb']

In [ ]:
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as font_manager
import marsilea as ma
import marsilea.plotter as mp
# plt.rcParams['figure.dpi'] = 150
font_dirs = ["/public/home/mowp/test/fonts/"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
plt.rcParams["font.family"] = "Arial"
sns.despine(top=True, right=True)
from itertools import product
from functools import reduce
import patchworklib as pw
from jpy_tools.otherTools import pwRecoverSeaborn, pwStack, F, pwShow
pw.show = pwShow
fc_recoverSns = pwRecoverSeaborn()
import seaborn.objects as so
from cycler import cycler
dt_snsStyle = {**sns.axes_style('ticks'), "legend.frameon": False, "axes.spines.top":False, "axes.spines.right":False, "axes.prop_cycle":cycler(color=['grey'])}
so.Plot.config.theme.update(dt_snsStyle)

<Figure size 640x480 with 0 Axes>

In [ ]:
import scanpy as sc
import muon as mu
import pandas as pd
import numpy as np
import scipy.sparse as ss
from tqdm import tqdm
from loguru import logger

In [ ]:
import pyranges as pr

In [ ]:
import utils

<Figure size 640x480 with 0 Axes>

In [ ]:
from jpy_tools import singleCellTools
from functools import partial

Global seed set to 0


In [ ]:
import seaborn.objects as so

dt_snsStyle = {
    **sns.axes_style("ticks"),
    "axes.spines.top": False,
    "axes.spines.right": False,
}

In [ ]:
dir_result = "/data/Zhaijx/liuzj/projects/circadian/03_analysis/"
dir_temp = "/data/Zhaijx/liuzj/projects/singleCell/02_jupyter/circadian/20221122/"

In [ ]:
# md = loadPkl('md_20220829_after_diffxpy_marker', lambda **dt:mu.read_h5mu(**dt), arg_path='filename', dir_path=dir_temp)
# ad = md['all']

In [ ]:
ls_sample = [8, 10, 12, 14, 16, 18, 20, 22, 24, 2, 4, 6] | F(map, str) | F(list)

In [ ]:
dt_sample2Phase = {x: y for x, y in zip(ls_sample, range(0, 26, 2))}
dt_phase2Sample = {y: x for x, y in dt_sample2Phase.items()}

In [ ]:
dt_sample2Ct = {x: y for x, y in zip(ls_sample, range(48, 72, 2))}

In [ ]:
# ad.obs['CT'] = ad.obs['Sample'].map(dt_sample2Ct)

In [ ]:
# ls_cluster = ad.obs['leiden'].cat.categories.to_list()

In [ ]:
# toPkl(md, 'md_20220825', 'ipf', dir_path=dir_temp)

In [ ]:
!mkdir {dir_temp}

In [ ]:
from dataclasses import dataclass
from typing import Any

import numpy as np
import matplotlib as mpl
from matplotlib.artist import Artist
from seaborn._core.scales import Scale
from seaborn._marks.base import (
    Mark,
    Mappable,
    MappableBool,
    MappableColor,
    MappableFloat,
    MappableStyle,
    resolve_properties,
    resolve_color,
    document_properties,
)
from collections import defaultdict


class BarBase(Mark):
    def _make_patches(self, data, scales, orient):
        kws = self._resolve_properties(data, scales)
        if orient == "x":
            kws["x"] = (data["x"] - data["width"] / 2).to_numpy()
            kws["y"] = data["baseline"].to_numpy()
            kws["w"] = data["width"].to_numpy()
            kws["h"] = (data["y"] - data["baseline"]).to_numpy()
        else:
            kws["x"] = data["baseline"].to_numpy()
            kws["y"] = (data["y"] - data["width"] / 2).to_numpy()
            kws["w"] = (data["x"] - data["baseline"]).to_numpy()
            kws["h"] = data["width"].to_numpy()

        kws.pop("width", None)
        kws.pop("baseline", None)

        val_dim = {"x": "h", "y": "w"}[orient]
        bars, vals = [], []

        for i in range(len(data)):
            row = {k: v[i] for k, v in kws.items()}

            # Skip bars with no value. It's possible we'll want to make this
            # an option (i.e so you have an artist for animating or annotating),
            # but let's keep things simple for now.
            if not np.nan_to_num(row[val_dim]):
                continue

            bar = mpl.patches.Rectangle(
                xy=(row["x"], row["y"]),
                width=row["w"],
                height=row["h"],
                facecolor=row["facecolor"],
                edgecolor=row["edgecolor"],
                linestyle=row["edgestyle"],
                linewidth=row["edgewidth"],
                **self.artist_kws,
            )
            bars.append(bar)
            vals.append(row[val_dim])

        return bars, vals

    def _resolve_properties(self, data, scales):
        resolved = resolve_properties(self, data, scales)

        resolved["facecolor"] = resolve_color(self, data, "", scales)
        resolved["edgecolor"] = resolve_color(self, data, "edge", scales)

        fc = resolved["facecolor"]
        if isinstance(fc, tuple):
            resolved["facecolor"] = fc[0], fc[1], fc[2], fc[3] * resolved["fill"]
        else:
            fc[:, 3] = fc[:, 3] * resolved["fill"]  # TODO Is inplace mod a problem?
            resolved["facecolor"] = fc

        return resolved

    def _legend_artist(
        self,
        variables,
        value: Any,
        scales,
    ) -> Artist:
        # TODO return some sensible default?
        key = {v: value for v in variables}
        key = self._resolve_properties(key, scales)
        artist = mpl.patches.Patch(
            facecolor=key["facecolor"],
            edgecolor=key["edgecolor"],
            linewidth=key["edgewidth"],
            linestyle=key["edgestyle"],
        )
        return artist


@dataclass
class Box(BarBase):
    """
    An oriented rectangular mark drawn between min/max values.
    Examples
    --------
    .. include:: ../docstrings/objects.Box.rst
    """

    color: MappableColor = Mappable("C0")
    alpha: MappableFloat = Mappable(0.7)
    fill: MappableBool = Mappable(True)
    edgecolor: MappableColor = Mappable(depend="color")
    edgealpha: MappableFloat = Mappable(1)
    edgewidth: MappableFloat = Mappable(rc="patch.linewidth")
    edgestyle: MappableFloat = Mappable("-")
    width: MappableFloat = Mappable(0.8, grouping=False)

    def _clip_edges(self, artist: Artist, ax):
        """Add a clip path to patch artist so that edges do not extend past face."""
        # This is a hack to handle the fact that the edge lines are centered on
        # the actual extents of the patch and overlap when stacking or dodging.
        # We may discover that this causes issues and needs to be revisited.

        # Because we are clipping, the edges end up looking half as wide as they
        # actually are. I don't love this clumsy workaround, which is going to
        # cause surprises if you work with the artists directly.
        artist.set_linewidth(artist.get_linewidth() * 2)
        linestyle = artist.get_linestyle()
        if linestyle[1]:
            linestyle = (linestyle[0], tuple(x / 2 for x in linestyle[1]))
        artist.set_linestyle(linestyle)

        # It should be faster to clip with a bbox than a path, but I cant't work
        # out how to get the intersection with the axes bbox.
        artist.set_clip_path(artist.get_path(), artist.get_transform() + ax.transData)
        if self.artist_kws.get("clip_on", True):
            # It seems the above hack undoes the default axes clipping
            artist.set_clip_box(ax.bbox)

    def _plot(self, split_gen, scales, orient):
        patches = defaultdict(list)

        for keys, data, ax in split_gen():
            kws = {}

            resolved = self._resolve_properties(keys, scales)

            kws["facecolor"] = resolved["facecolor"]
            kws["edgecolor"] = resolved["edgecolor"]
            kws["linewidth"] = resolved["edgewidth"]
            kws["linestyle"] = resolved["edgestyle"]

            other = {"x": "y", "y": "x"}[orient]

            if not set(data.columns) & {f"{other}min", f"{other}max"}:
                agg = {f"{other}min": (other, "min"), f"{other}max": (other, "max")}
                data = data.groupby([orient, "width"]).agg(**agg).reset_index()

            for row in data.itertuples():
                if orient == "x":
                    verts = np.array(
                        [
                            row.x - row.width / 2,
                            row.ymin,
                            row.x + row.width / 2,
                            row.ymin,
                            row.x + row.width / 2,
                            row.ymax,
                            row.x - row.width / 2,
                            row.ymax,
                        ]
                    ).reshape((4, 2))
                else:
                    verts = np.array(
                        [
                            row.xmin,
                            row.y - row.width / 2,
                            row.xmax,
                            row.y - row.width / 2,
                            row.xmax,
                            row.y + row.width / 2,
                            row.xmin,
                            row.y + row.width / 2,
                        ]
                    ).reshape((4, 2))

                patches[ax].append(mpl.patches.Polygon(verts, **kws))
                ax.update_datalim(verts)

        for ax, ax_patches in patches.items():
            for patch in ax_patches:
                self._clip_edges(patch, ax)
                ax.add_patch(patch)

In [ ]:
ad = loadPkl('ad_scshc', lambda **dt:sc.read_h5ad(**dt), arg_path='filename', dir_path=dir_temp)

In [ ]:
pad = singleCellTools.plotting.PlotAnndata(ad, 'sx')

In [ ]:
# pad.getPb("CT").write_h5ad("/data/Zhaijx/qinyw/data/flsnRNA/illumina_lib/circadian/20231110/soupX_all.h5ad")

In [ ]:
# pad.getPb(["CT", "leiden_subcluster"]).write_h5ad("/data/Zhaijx/qinyw/data/flsnRNA/illumina_lib/circadian/20231110/soupX_eachCluster.h5ad")

# load data

## gene name locus 

In [ ]:
df_gff3 = (
    pr.read_gff3("/data/Zhaijx/liuzj/data/Araport11/gene.gff", as_df=True)
    .query("Feature == 'gene'")
    .set_index("ID")
)

In [ ]:
dt_locus2Name = df_gff3["Alias"].dropna().to_dict()
dt_locus2Name.update(df_gff3["symbol"].dropna().to_dict())

## core clock gene

In [ ]:
ls_coreGenes = [
    "AT2G46830",
    "AT1G01060",
    "AT1G12910",
    "AT3G26640",
    "AT2G46790",
    "AT5G02810",
    "AT3G09600",
    "AT5G52660",
    "AT5G02840",
    "AT5G64170",
    "AT3G54500",
    "AT5G24470",
    "AT5G60100",
    "AT5G61380",
    "AT5G08330",
    "AT1G22770",
    "AT5G59570",
    "AT3G46640",
    "AT2G40080",
    "AT2G25930",
]
ls_coreGeneNames = [
    "CCA1",
    "LHY",
    "LWD1",
    "LWD2",
    "PRR9",
    "PRR7",
    "RVE8",
    "RVE6",
    "RVE4",
    "LNK1",
    "LNK2",
    "PRR5",
    "PRR3",
    "TOC1",
    "CHE",
    "GI",
    "NOX",
    "LUX",
    "ELF4",
    "ELF3",
]
dt_coreGene = dict(zip(ls_coreGenes, ls_coreGeneNames))
dt_coreGene

# added
dt_coreGene['AT5G57360'] = 'ZTL'

In [ ]:
_ls = """AT5G64170
AT5G61380
AT5G60100
AT5G57360
AT5G52660
AT5G24470
AT5G02840
AT5G02810
AT3G54500
AT3G46640
AT3G09600
AT2G46830
AT2G46790
AT2G40080
AT2G25930
AT1G22770
AT1G01060""".split()
dt_coreGene = {x:dt_coreGene[x] for x in _ls}
dt_coreGene

{'AT5G64170': 'LNK1',
 'AT5G61380': 'TOC1',
 'AT5G60100': 'PRR3',
 'AT5G57360': 'ZTL',
 'AT5G52660': 'RVE6',
 'AT5G24470': 'PRR5',
 'AT5G02840': 'RVE4',
 'AT5G02810': 'PRR7',
 'AT3G54500': 'LNK2',
 'AT3G46640': 'LUX',
 'AT3G09600': 'RVE8',
 'AT2G46830': 'CCA1',
 'AT2G46790': 'PRR9',
 'AT2G40080': 'ELF4',
 'AT2G25930': 'ELF3',
 'AT1G22770': 'GI',
 'AT1G01060': 'LHY'}

# QC

In [ ]:
from joblib import Parallel, delayed

In [ ]:
ls_sample = [8, 10, 12, 14, 16, 18, 20, 22, 24, 2, 4, 6] | F(map, str) | F(list)

In [ ]:
def removeDoubletsAndGetAmbientScore(sample):
    import rpy2.robjects as ro
    ro.r(".libPaths")("/public1/software/liuzj/softwares/anaconda3/envs/sc_py/lib/R/library")
    ad_sample = sc.read_10x_h5(
        f"/public/home/liuzj/projects/circadian/02_cellranger/circadian_{sample}/outs/filtered_feature_bc_matrix.h5"
        )
    ad_sample.layers["raw"] = ad_sample.X.copy()
    singleCellTools.detectDoublet.byScDblFinder(ad_sample)
    ad_sampleRaw = sc.read_10x_h5(
        f"/public/home/liuzj/projects/circadian/02_cellranger/circadian_{sample}/outs/raw_feature_bc_matrix.h5"
    )
    ad_sampleRaw.layers["raw"] = ad_sampleRaw.X.copy()
    singleCellTools.removeAmbient.removeAmbientBySoupx(ad_sample, ad_sampleRaw, res=1.5)
    return ad_sample

In [ ]:
ls_afterAmbAndRemoveDoubletsAd = Parallel(n_jobs=12, backend='multiprocessing')(delayed(removeDoubletsAndGetAmbientScore)(sample) for sample in ls_sample)

In [ ]:
ad = sc.concat(dict(zip(ls_sample,ls_afterAmbAndRemoveDoubletsAd)), join='outer', label='Sample', index_unique='-')

In [ ]:
def _getCt(x):
    x = int(x)
    x -= 8
    if x < 0:
        x += 24
    return str(x)

In [ ]:
ad.obs['CT'] = ad.obs['Sample'].map(_getCt).astype('category')

In [ ]:
(
    so.Plot(ad.obs, x='ambientRnaFractionEstimatedBySoupx')
    .facet('CT', wrap=4)
    .share(x=False)
    .add(so.Bars(), so.Hist(bins=100))
    .label(x='RHO')
    .scale(x=so.Continuous().tick(every=0.1))
    .limit(x=[0, 1])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
singleCellTools.basic.initLayer(ad, layer='raw')

In [ ]:
sc.pp.filter_genes(ad, min_cells=10)

In [ ]:
singleCellTools.plotting.plotCellScatter(ad, batch='Sample')

In [ ]:
(
    so.Plot(ad.obs, x='n_genes')
    .facet('CT', wrap=4)
    .share(y=False, x=False)
    .add(so.Bars(), so.Hist(bins=100))
    .label(x='Gene counts')
    .scale(x=so.Continuous().tick(every=500))
    .limit(x=[0, 3000])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
(
    so.Plot(ad.obs, x='n_counts')
    .facet('CT', wrap=4)
    .share(y=False, x=False)
    .add(so.Bars(), so.Hist(bins=100))
    .label(x='UMI counts')
    .scale(x=so.Continuous().tick(every=1000))
    .limit(x=[0, 5000])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
(
    so.Plot(ad.obs, x='percent_ct')
    .facet('CT', wrap=4)
    .share(y=False, x=False)
    .add(so.Bars(), so.Hist(bins=100))
    .label(x='Percent CT')
    .scale(x=so.Continuous().tick(every=0.02))
    .limit(x=[0, 0.1])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
def addDensityToDf(df:pd.DataFrame, x:str, y:str, group=None, bins=20):
    from scipy.interpolate import interpn
    from scipy.stats import gaussian_kde

    if group:
        df = df.groupby(group, as_index=False).apply(lambda df: addDensityToDf(df, x=x, y=y, bins=bins)).reset_index(level=0, drop=True)
    elif bins is None:
        x = df[x].values
        y = df[y].values
        xy = np.vstack([x,y])
        z = gaussian_kde(xy)(xy)
        df['temp_density'] = z
        df = df.sort_values('temp_density')
    else:
        x = df[x].values
        y = df[y].values
        data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
        z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)
        z[np.where(np.isnan(z))] = 0.0
        df['temp_density'] = z
        df = df.sort_values('temp_density')
    return df

In [ ]:
_df = addDensityToDf(ad.obs, x='n_counts', y='percent_ct', group='CT', bins=50)
(
    so.Plot(_df, x='n_counts', y='percent_ct', color='temp_density')
    .facet('CT', wrap=4)
    .share(y=False, x=False)
    .add(so.Dots(pointsize=1, fillalpha=0.2, alpha=0.2), legend=False)
    .label(x='UMI counts', y='Percent CT')
    # .scale(x=so.Continuous().tick(every=1000))
    .limit(x=[0, 5000], y=[0,0.1])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
_df = addDensityToDf(ad.obs, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', group='CT', bins=50)
(
    so.Plot(_df, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', color='temp_density')
    .facet('CT', wrap=4)
    .share(y=False, x=False)
    .add(so.Dots(pointsize=1, fillalpha=0.2, alpha=0.2), legend=False)
    .label(x='UMI counts', y='RHO')
    # .scale(x=so.Continuous().tick(every=1000))
    # .limit(x=[0, 5000])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
_df = addDensityToDf(ad.obs, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', group='CT', bins=50)
(
    so.Plot(_df, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', color='temp_density')
    .facet('CT', wrap=4)
    .add(so.Dots(pointsize=1, fillalpha=0.2, alpha=0.2), legend=False)
    .label(x='UMI counts', y='RHO')
    # .scale(x=so.Continuous().tick(every=1000))
    # .limit(x=[0, 5000])
    .layout(size=(12,6))
    .theme(dt_snsStyle)
)

In [ ]:
_df = addDensityToDf(ad.obs, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', group='CT', bins=50)
_ls = ['3', '8', '12', '13']
_df['Cluster'] = _df['Cluster'].map(lambda x: x if x in _ls else 'Other')
_df = _df.sort_values("Cluster", ascending=False)
(
    so.Plot(_df, x='n_counts', y='ambientRnaFractionEstimatedBySoupx', color='Cluster')
    .facet('CT', wrap=4)
    .add(so.Dots(pointsize=1, fillalpha=1, alpha=1),)
    .label(x='UMI counts', y='RHO')
    # .scale(x=so.Continuous().tick(every=1000))
    # .limit(x=[0, 5000])
    .scale(color={**singleCellTools.basic.getadataColor(ad, 'Cluster'), 'Other': 'grey'})
    .layout(size=(12,6))
    .theme({**dt_snsStyle, 'legend.markerscale': 5})
)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * M.mad()) | (
        np.median(M) + nmads * M.mad() < M
    )
    return outlier

In [ ]:
ls_outlier = []
for _ad in singleCellTools.basic.splitAdata(ad, 'CT', copy=False):
    ls_outlier.append(is_outlier(_ad, 'n_counts', 5) | is_outlier(_ad, "n_genes", 5) | is_outlier(_ad, "percent_ct", 5))

In [ ]:
ad.obs["outlier"] = pd.concat(ls_outlier)

In [ ]:
ad = ad[~ad.obs['outlier']]

In [ ]:
ad.obs.eval("percent_ct >= 0.1").value_counts()

In [ ]:
sc.pp.filter_genes(ad, min_cells=10)

In [ ]:
ad.raw = ad

In [ ]:
ad = ad[:, ~ad.var.index.str.contains(r'ATCG|ATMG')]

In [ ]:
ad.obs['Batch'] = np.where((ad.obs['Sample'].astype(int) % 4) == 0, '1', '2')

In [ ]:
toPkl(ad, 'ad_afterQC_1123', 'ipf', dir_path=dir_temp)

In [ ]:
ad = loadPkl(
    "ad_afterQC_1123",
    lambda **dt: sc.read_h5ad(**dt),
    arg_path="filename",
    dir_path=dir_temp,
)

In [ ]:
ad.obs = ad.obs.assign(
    CT=lambda x: x["CT"].map(lambda y: y >> F(int) >> F(lambda z: z + 24) >> F(str))
)

In [ ]:
ad.obs["n_counts"] = ad.layers["raw"].sum(1).A1

In [ ]:
ad.layers["sx"] = ad.layers["soupX_corrected"].copy()

In [ ]:
ad.layers["sx_log"] = ad.layers["sx"].copy()
sc.pp.normalize_total(ad, 1e4, layer="sx_log")
sc.pp.log1p(ad, layer="sx_log", base=2)

In [ ]:
_dt = {**dt_snsStyle, "axes.spines.left": False}
g0 = (
    so.Plot(ad.obs, color="CT")
    .add(so.Bar(fill=False), so.Count(), y="CT", legend=False)
    .theme(_dt)
    .label(x="Nuclei count", y="")
    .scale(color=sc.plotting.palettes.default_20)
    .on(pw.basefigure)
    .plot()
)
bk0 = pw.load_seabornobj(g0, figsize=(1, 3))
bk0.invert_xaxis()
bk0.yaxis.set_label_position("right")
bk0.yaxis.set_ticks_position("right")
bk0.set_yticklabels([f"CT {x.get_text()}" for x in bk0.get_yticklabels()])
bk0.yaxis.set_ticks_position("none")
for axis in ["top", "bottom", "left", "right"]:
    bk0.spines[axis].set_linewidth(2)
bk0.tick_params(width=2)
bk0.set_xlabel("Nucleus count", fontsize=14)
bk0.set_xscale("symlog")
bk0.set_xlim(3e4, 0)
bk0.set_xticklabels(
    [
        x.get_text() if x.get_text() != "$\\mathdefault{0}$" else ""
        for x in bk0.get_xticklabels()
    ],
    fontsize=12,
)
bk0.tick_params(width=2)

with plt.rc_context(_dt):
    bk1 = pw.Brick(figsize=(5, 3))

    sns.boxplot(ad.obs, x="n_genes", y="CT", showfliers=False, ax=bk1)
    for i, (box, color) in enumerate(
        zip(
            [x for x in bk1.get_children() if isinstance(x, mpl.patches.PathPatch)],
            sc.plotting.palettes.default_20,
        )
    ):
        box.set_edgecolor(color)
        box.set_facecolor("white")
        # iterate over whiskers and median lines
        for j in range(5 * i, 5 * (i + 1)):
            bk1.lines[j].set_color(color)
    bk1.yaxis.set_ticks_position("none")
    bk1.set_yticklabels([f"" for x in bk1.get_yticklabels()])
    bk1.set_xticklabels([x for x in bk1.get_xticklabels()], fontsize=12)
    bk1.set_ylabel("")
    bk1.set_xlabel("Gene count", fontsize=14)
    plt.close()

for axis in ["top", "bottom", "left", "right"]:
    bk1.spines[axis].set_linewidth(2)
bk1.tick_params(width=2)

bk = pwStack([bk0, bk1], wmargin=0)
pwShow(bk)